In [1]:
import os
import pickle
import numpy as np
from PIL import Image
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize Mediapipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Prepare Data
data_dir = './data2'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

data = {}

# Extract Pose Landmarks
def extract_landmarks(image_path):
    img = Image.open(image_path)
    img_rgb = img.convert('RGB')
    results = pose.process(np.array(img_rgb))
    if results.pose_landmarks:
        return np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.pose_landmarks.landmark]).flatten()
    else:
        return None

for j, dir_ in enumerate([train_dir, val_dir]):
    features = []
    labels = []
    for category in os.listdir(dir_):
        category_path = os.path.join(dir_, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            pose_features = extract_landmarks(img_path)
            if pose_features is not None:
                features.append(pose_features)
                labels.append(category)

    if j == 0:
        data['training_data'] = features
        data['training_labels'] = labels
    else:
        data['validation_data'] = features
        data['validation_labels'] = labels

# Train Model
model = RandomForestClassifier(random_state=0)
model.fit(data['training_data'], data['training_labels'])

# Evaluate Model
y_pred = model.predict(data['validation_data'])
score = accuracy_score(y_pred, data['validation_labels'])
print(score)

# Save the Model
with open('exercise_model.pkl', 'wb') as f:
    pickle.dump(model, f)

I0000 00:00:1730368819.992031 10553129 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


0.8172043010752689


In [2]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', probability=True, random_state=0)
svm_model.fit(data['training_data'], data['training_labels'])

# Evaluate SVM Model
y_pred_svm = svm_model.predict(data['validation_data'])
score_svm = accuracy_score(y_pred_svm, data['validation_labels'])
print(f'SVM Model Accuracy: {score_svm}')

# Save the SVM Model
with open('exercise_svm_model.pkl', 'wb') as f:
    pickle.dump(svm_model, f)

SVM Model Accuracy: 0.7526881720430108


In [3]:
from sklearn.neural_network import MLPClassifier

# Train Neural Network Model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=0)
nn_model.fit(data['training_data'], data['training_labels'])

# Evaluate Neural Network Model
y_pred_nn = nn_model.predict(data['validation_data'])
score_nn = accuracy_score(y_pred_nn, data['validation_labels'])
print(f'Neural Network Model Accuracy: {score_nn}')

# Save the Neural Network Model
with open('exercise_nn_model.pkl', 'wb') as f:
    pickle.dump(nn_model, f)

Neural Network Model Accuracy: 0.8279569892473119


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
